In [4]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
#style.use("ggplot")
#from sklearn.svm import SVR
from sklearn import svm

In [5]:
##Functions
def printMetrics(test, predictions):
    print(f"Score: {explained_variance_score(test, predictions):.2f}")
    print(f"MAE: {mean_absolute_error(test, predictions):.2f}")
    print(f"RMSE: {math.sqrt(mean_squared_error(test, predictions)):.2f}")
    print(f"r2: {r2_score(test, predictions):.2f}")

#def  calculateScore(F_Features,F_Target):
#    X_train, X_test, y_train, y_test = train_test_split(F_Features, F_Target, test_size=0.25, random_state=1)
#    reg = ElasticNet()
#    reg.fit(X_train, y_train)
#    predictions = reg.predict(X_test)
#    printMetrics(y_test, predictions)

    
def  predictValue(F_Features,F_Target,predict_df):
    X_train, X_test, y_train, y_test = train_test_split(F_Features, F_Target, test_size=0.25, random_state=1)
    reg = ElasticNet()
    reg.fit(X_train, y_train)
    predictions = reg.predict(predict_df)
    return predictions

In [6]:
# set variables:
country = "CAN" 
unwanted_columns = ["continent",
                   "location",
                   "new_cases_smoothed",
                   "new_deaths_smoothed",
                   "new_cases_smoothed_per_million",
                   "new_deaths_smoothed_per_million",
                   "new_tests_smoothed",
                   "new_tests_smoothed_per_thousand",
                   "new_vaccinations_smoothed",
                   "new_vaccinations_smoothed_per_million"]
#drop_column_if_NAN = ["total_deaths"]

Preparing Data

In [7]:
# import dataset
covid19DataAll = pd.read_csv("https://raw.githubusercontent.com/WhipSnake23/Python-Class-Project/main/Data/owid-covid-data.csv")

# filter dataset for a country
covid19DataCountry = covid19DataAll[(covid19DataAll["iso_code"] == country)]
# covid19DataCountry # For Testing Only

# drop unwanted_columns
covid19Data = covid19DataCountry.drop(unwanted_columns, axis=1)

# reset index
covid19Data.reset_index(drop=True, inplace=True)
covid19Data.dropna(subset=['total_deaths'],inplace=True) # drop only if NaN in specific columns
covid19Data

,iso_code,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,new_deaths_per_million,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
43,CAN,2020-03-09,77.0,13.0,1.0,1.0,2.040,0.344,0.026,0.026,...,44017.591,0.5,105.599,7.37,12.0,16.6,NaN,2.5,82.43,0.929
44,CAN,2020-03-10,79.0,2.0,1.0,0.0,2.093,0.053,0.026,0.000,...,44017.591,0.5,105.599,7.37,12.0,16.6,NaN,2.5,82.43,0.929
45,CAN,2020-03-11,108.0,29.0,1.0,0.0,2.862,0.768,0.026,0.000,...,44017.591,0.5,105.599,7.37,12.0,16.6,NaN,2.5,82.43,0.929
46,CAN,2020-03-12,117.0,9.0,1.0,0.0,3.100,0.238,0.026,0.000,...,44017.591,0.5,105.599,7.37,12.0,16.6,NaN,2.5,82.43,0.929
47,CAN,2020-03-13,193.0,76.0,1.0,0.0,5.114,2.014,0.026,0.000,...,44017.591,0.5,105.599,7.37,12.0,16.6,NaN,2.5,82.43,0.929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388,CAN,2021-02-17,839155.0,2561.0,21439.0,44.0,22233.891,67.855,568.039,1.166,...,44017.591,0.5,105.599,7.37,12.0,16.6,NaN,2.5,82.43,0.929
389,CAN,2021-02-18,842590.0,3435.0,21509.0,70.0,22324.903,91.012,569.893,1.855,...,44017.591,0.5,105.599,7.37,12.0,16.6,NaN,2.5,82.43,0.929
390,CAN,2021-02-19,845673.0,3083.0,21581.0,72.0,22406.589,81.686,571.801,1.908,...,44017.591,0.5,105.599,7.37,12.0,16.6,NaN,2.5,82.43,0.929
391,CAN,2021-02-20,848330.0,2657.0,21631.0,50.0,22476.988,70.399,573.126,1.325,...,44017.591,0.5,105.599,7.37,12.0,16.6,NaN,2.5,82.43,0.929


In [8]:
featuresColumns = ["total_cases"]
#featureColumns = ["new_cases", "median_age","population_density","population","hosp_patients","total_cases","positive_rate"]
X=covid19Data[featuresColumns]
print(X)

     total_cases
43          77.0
44          79.0
45         108.0
46         117.0
47         193.0
..           ...
388     839155.0
389     842590.0
390     845673.0
391     848330.0
392     850737.0

[350 rows x 1 columns]


In [9]:
targetColumns = "total_deaths"
y=covid19Data[targetColumns]
print(y)

43         1.0
44         1.0
45         1.0
46         1.0
47         1.0
        ...   
388    21439.0
389    21509.0
390    21581.0
391    21631.0
392    21675.0
Name: total_deaths, Length: 350, dtype: float64


In [10]:
clf = svm.SVC(kernel='linear', C = 1.0)
clf.fit(X,y)

SVC(kernel='linear')

In [11]:
print(clf.score(X, y))     

1.0
